In [1]:
import pandas as pd
import numpy as np
import mat73


In [2]:
#load the dataset
data_dict = mat73.loadmat('./JF2_pubdata.mat')

In [3]:
#convert matlib datenum to python datetime

from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import datetime
from datetime import timedelta

def datenum_to_datetime(datenum):
    """
    Convert Matlab datenum into Python datetime.
    :param datenum: Date in datenum format
    :return:        Datetime object corresponding to datenum.
    """
    days = datenum % 1
    hours = days % 1 * 24
    minutes = hours % 1 * 60
    seconds = minutes % 1 * 60
    return datetime.fromordinal(int(datenum)) \
           + timedelta(days=int(days)) \
           + timedelta(hours=int(hours)) \
           + timedelta(minutes=int(minutes)) \
           + timedelta(seconds=round(seconds)) \
           - timedelta(days=366)

year_list=[]
month_list=[]
for i in  data_dict['unique_dates']:
    yyyy=datenum_to_datetime(i).year
    mm=datenum_to_datetime(i).month
    year_list.append(yyyy)
    month_list.append(mm)

yq_list=[]
for (m,y) in zip(month_list,year_list):
    if np.isin(m,(1,2,3)):
        yq=y*100+1
    elif np.isin(m,(4,5,6)):
        yq=y*100+2
    elif np.isin(m,(7,8,9)):
        yq=y*100+3
    elif np.isin(m,(10,11,12)):
        yq=y*100+4
    yq_list.append(yq)

In [4]:
#calculate target, the DTS scaled excess return
data_dict['EXCESS_RETURN']
data_dict['DTS'][data_dict['DTS']<0.25]=0.25
target=data_dict['EXCESS_RETURN']/data_dict['DTS']
target.shape

(8759, 206)

In [5]:
final_data=np.hstack((data_dict['CHARS'][:,:-1,:],np.expand_dims(target,1))) #shape=[#bonds,#variables,#month][8759,30,206]

In [6]:
final_data.shape

(8759, 30, 206)

In [7]:
chunk = final_data[14,:,:]
chunk = chunk.transpose()
chunk[139,:]

array([ 0.17480211,  0.31596306, -0.35158311,  0.14841689, -0.4368561 ,
        0.40699208, -0.08905013,  0.47163588, -0.11464863, -0.34823491,
       -0.28866667, -0.29023747, -0.03364116,  0.1207124 , -0.27110818,
       -0.30277045,  0.29221636,  0.22031662,  0.09366755, -0.37660838,
        0.05013193, -0.27612434,  0.39541405, -0.04287599,  0.07585752,
        0.24208443,  0.43205805,  0.28693931, -0.42084433, -0.01345372])

In [8]:
non_missing_indices = (np.argwhere(~np.isnan(chunk))[:,0])
print(non_missing_indices[0])
print(non_missing_indices[-1])

97
205


In [47]:
bond_list = []
for i in range(final_data.shape[0]):
    chunk = final_data[i,:,:]
    chunk = chunk.transpose()
    #print(len(set(np.argwhere(~np.isnan(chunk))[:,0])))
    if len(set(np.argwhere(~np.isnan(chunk))[:,0])) > 150:
        bond_list.append(i)

In [48]:
len(bond_list)


42

In [12]:
chunk = final_data[708,:,:]
chunk = chunk.transpose()
missing_indices = list(set((np.argwhere(np.isnan(chunk))[:,0])))
all_indices = list(np.arange(0,206))
all_indices
non_missing_indices = [x for x in all_indices if x not in missing_indices]
non_missing_indices
data = chunk[[non_missing_indices],:]
data
missing_values_mask = np.isnan(data) | (data is None)
# Count the number of missing values
num_missing_values = np.sum(missing_values_mask)
print(num_missing_values)
data = data.reshape((data.shape[1],data.shape[2]))
print(data.shape)



0
(186, 30)


In [14]:
for i in bond_list:
    chunk = final_data[i,:,:]
    chunk = chunk.transpose()
    missing_indices = list(set((np.argwhere(np.isnan(chunk))[:,0])))
    all_indices = list(np.arange(0,206))
    all_indices
    non_missing_indices = [x for x in all_indices if x not in missing_indices]
    non_missing_indices
    data = chunk[[non_missing_indices],:]
    data
    missing_values_mask = np.isnan(data) | (data is None)
    # Count the number of missing values
    num_missing_values = np.sum(missing_values_mask)
    print(data.shape)

(1, 165, 30)
(1, 186, 30)
(1, 152, 30)
(1, 161, 30)
(1, 164, 30)
(1, 197, 30)
(1, 188, 30)
(1, 181, 30)
(1, 197, 30)
(1, 178, 30)
(1, 171, 30)
(1, 169, 30)
(1, 158, 30)
(1, 206, 30)
(1, 152, 30)
(1, 152, 30)
(1, 152, 30)
(1, 152, 30)
(1, 206, 30)
(1, 196, 30)
(1, 153, 30)
(1, 161, 30)
(1, 191, 30)
(1, 202, 30)
(1, 206, 30)
(1, 205, 30)
(1, 198, 30)
(1, 184, 30)
(1, 171, 30)
(1, 155, 30)
(1, 172, 30)
(1, 173, 30)
(1, 195, 30)
(1, 163, 30)
(1, 205, 30)
(1, 206, 30)
(1, 168, 30)
(1, 205, 30)
(1, 206, 30)
(1, 169, 30)
(1, 175, 30)
(1, 195, 30)


In [15]:
for i in bond_list:
    chunk = final_data[i,:,:]
    chunk = chunk.transpose()
    non_missing_indices = (np.argwhere(~np.isnan(chunk))[:,0])
    start = non_missing_indices[0]
    end = non_missing_indices[-1]
    my_array = chunk[start:end+1,:]
    # Check for missing values (NaN or None)
    missing_values_mask = np.isnan(my_array) | (my_array is None)

    # Count the number of missing values
    num_missing_values = np.sum(missing_values_mask)
    print(num_missing_values)

0
180
0
600
30
0
540
540
0
450
0
0
0
0
1620
1620
1620
1620
0
0
0
0
60
0
0
0
240
0
0
0
990
60
300
0
0
0
0
0
0
0
0
0


In [16]:
c = list(data_dict['unique_charnames'])
c.pop()
c.append('target')
c

['BOND_AGE',
 'BOND_COUPON',
 'BOND_FACE_VALUE',
 'BP',
 'DEBT_OVER_EBITDA',
 'DURATION',
 'EMOM_6M',
 'EP',
 'EQUITY_MKT_CAP',
 'EQUITY_VOL',
 'FIRM_TOTAL_DEBT',
 'MOM_6M',
 'MOM_6M_INDUSTRY_LEVEL',
 'MOM_6M_INTERACTED_RATINGS',
 'NEG_BOOK_LEVERAGE',
 'NEG_MKT_LEVERAGE',
 'NEG_VOL_OF_TURNOVER',
 'OAS',
 'OPERATING_LEVERAGE',
 'PROFITABILITY',
 'PROFITABILITY_5YR_CHANGE',
 'RATING',
 'SHUMWAY_D2D',
 'SKEW',
 'SPREAD_MOM_6M',
 'SPREAD_PD_RATIO',
 'TOTAL_VOL',
 'VIX_BETA',
 'VALUE_AT_RISK',
 'target']

In [17]:
chunk = final_data[13,:,:]
chunk = chunk.transpose()
non_missing_indices = (np.argwhere(~np.isnan(chunk))[:,0])
start = non_missing_indices[0]
end = non_missing_indices[-1]
print(chunk[start:end+1,:],start,end)
data = chunk[start:end+1,:]

[[-0.4864486  -0.43831776 -0.43878505 ...  0.47102804  0.25981308
  -0.03717068]
 [-0.45022624 -0.42624434 -0.43936652 ...  0.46199095  0.19864253
   0.06512186]
 [-0.42559787 -0.42116918 -0.43711249 ...  0.45394154  0.1510186
  -0.03437261]
 ...
 [ 0.17480211 -0.22691293 -0.46701847 ...  0.27176781  0.44195251
   0.00258755]
 [ 0.19125326 -0.21703655 -0.46801567 ...  0.26109661  0.44451697
  -0.00378258]
 [ 0.19684003 -0.20967742 -0.46872943 ...  0.25905201  0.44996708
   0.002351  ]] 97 141


In [18]:
df = pd.DataFrame(data,columns = c)

In [19]:
df

,BOND_AGE,BOND_COUPON,BOND_FACE_VALUE,BP,DEBT_OVER_EBITDA,DURATION,EMOM_6M,EP,EQUITY_MKT_CAP,EQUITY_VOL,...,PROFITABILITY_5YR_CHANGE,RATING,SHUMWAY_D2D,SKEW,SPREAD_MOM_6M,SPREAD_PD_RATIO,TOTAL_VOL,VIX_BETA,VALUE_AT_RISK,target
0,-0.486449,-0.438318,-0.438785,-0.058879,-0.454206,-0.055140,-0.443925,0.342844,-0.144694,0.257944,...,-0.044554,-0.044049,-0.076977,-0.001869,-0.458879,0.038318,-0.312150,0.471028,0.259813,-0.037171
1,-0.450226,-0.426244,-0.439367,-0.067421,-0.453846,-0.078281,-0.360633,0.355072,-0.134798,0.333182,...,-0.046196,-0.043764,-0.202899,-0.032127,-0.039367,0.082805,-0.293665,0.461991,0.198643,0.065122
2,-0.425598,-0.421169,-0.437112,-0.112932,-0.425598,-0.098760,-0.260850,0.161348,-0.101950,0.251107,...,-0.117021,-0.049046,-0.079787,-0.202391,0.010186,0.040301,-0.258193,0.453942,0.151019,-0.034373
3,-0.401632,-0.408935,-0.439433,-0.091065,-0.426976,-0.108247,0.128007,0.165808,-0.101381,0.223368,...,-0.121668,-0.084589,-0.149892,-0.174399,-0.077320,0.045533,-0.242268,0.412371,0.130584,-0.001425
4,-0.385211,-0.405417,-0.441101,-0.077816,-0.429493,-0.121668,0.079174,0.155202,-0.098967,0.105331,...,-0.120482,-0.091986,-0.126345,-0.114789,-0.183577,0.078676,-0.229149,0.439811,0.134566,0.088352
5,-0.372203,-0.406196,-0.440620,-0.105682,-0.418244,-0.129948,-0.051635,0.122203,-0.080353,0.183305,...,-0.091731,-0.098361,-0.109651,-0.096386,0.330465,-0.053356,-0.192771,0.406196,0.139415,-0.001728
6,-0.356892,-0.408692,-0.442054,-0.094203,-0.420983,-0.140035,0.463549,0.145870,-0.085055,-0.058385,...,-0.094203,-0.102378,-0.095782,-0.078578,0.197981,0.006585,-0.204126,0.396400,0.125110,0.041710
7,-0.330450,-0.400519,-0.442474,-0.070749,-0.417820,-0.153114,0.319559,0.187446,-0.097228,0.163495,...,-0.100173,-0.099566,-0.131009,-0.155709,0.384083,-0.072664,-0.196367,0.367647,0.083910,-0.013698
8,-0.318298,-0.385957,-0.439574,0.099149,-0.420069,-0.157872,-0.284681,0.269889,-0.102045,0.413118,...,-0.014894,-0.088737,-0.102045,-0.062553,0.237872,-0.056596,-0.146809,0.369787,0.077872,-0.000374
9,-0.291561,-0.380169,-0.440084,0.127004,-0.417376,-0.168354,-0.199578,0.278296,-0.116392,0.110127,...,-0.006329,-0.083157,-0.121941,0.036709,0.375105,-0.081435,-0.138819,0.364979,0.094093,0.029923


In [20]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import glob

from RNN import RNN
#from CNN import CNN
from Transformer import Transformer

from utils import series_to_supervised

2023-07-26 19:55:04.178985: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-26 19:55:04.182404: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 19:55:04.247355: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 19:55:04.248475: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 19:55:05.044141: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [37]:

look_back = 8
n_features = df.shape[1] - 1

# Walk-forward data split to avoid data leakage
X_train, y_train, X_test, y_test, scale_X = series_to_supervised(df, train_size=0.7, n_in=look_back, n_out=7, target_column='target', dropnan=True, scale_X=True)



In [38]:
X_train.head(20)

,BOND_AGE(t-8),BOND_COUPON(t-8),BOND_FACE_VALUE(t-8),BP(t-8),DEBT_OVER_EBITDA(t-8),DURATION(t-8),EMOM_6M(t-8),EP(t-8),EQUITY_MKT_CAP(t-8),EQUITY_VOL(t-8),...,PROFITABILITY(t-1),PROFITABILITY_5YR_CHANGE(t-1),RATING(t-1),SHUMWAY_D2D(t-1),SKEW(t-1),SPREAD_MOM_6M(t-1),SPREAD_PD_RATIO(t-1),TOTAL_VOL(t-1),VIX_BETA(t-1),VALUE_AT_RISK(t-1)
8,0.000000,0.000000,0.947099,0.144163,0.000000,1.000000,0.000000,0.619445,0.000000,0.824820,...,0.032247,0.054935,0.760486,0.105006,0.251375,0.887270,0.718083,0.801546,0.604401,0.016226
9,0.053012,0.052805,0.928708,0.121380,0.009208,0.947517,0.091784,0.653777,0.092094,0.909759,...,0.070593,0.272883,0.806966,0.147312,0.359830,0.739532,0.747221,0.958591,0.612590,0.000000
10,0.089056,0.075002,1.000000,0.000000,0.732808,0.901069,0.201741,0.109897,0.397803,0.817102,...,0.071330,0.294771,0.830918,0.118251,0.475393,0.878198,0.702180,0.983911,0.594191,0.043592
11,0.124130,0.128512,0.926606,0.058319,0.697508,0.879551,0.630246,0.122419,0.403107,0.785786,...,0.079431,0.320801,0.798483,0.201866,0.502988,0.944654,0.608226,0.968979,0.550152,0.091248
12,0.148163,0.143897,0.873862,0.093656,0.633039,0.849112,0.576434,0.092644,0.425566,0.652532,...,0.177718,0.467327,0.788811,0.263168,0.612784,0.926671,0.548683,1.000000,0.528845,0.112560
13,0.167199,0.140489,0.889074,0.019335,0.921172,0.830332,0.432288,0.000000,0.598802,0.740558,...,0.147683,0.434394,0.756741,0.277589,0.596295,1.000000,0.321948,0.989433,0.504734,0.142735
14,0.189607,0.129574,0.843693,0.049951,0.851013,0.807455,1.000000,0.066444,0.555045,0.467709,...,0.173119,0.431028,0.758240,0.369948,0.679330,0.865551,0.425452,0.929375,0.459526,0.167158
15,0.228306,0.165320,0.830421,0.112505,0.932042,0.777791,0.841329,0.183168,0.441751,0.718194,...,0.260252,0.529171,0.758243,0.322080,0.720636,0.782918,0.444403,0.893803,0.396743,0.171604
16,0.246090,0.229007,0.922131,0.565630,0.874440,0.767000,0.175481,0.414625,0.396921,1.000000,...,0.212945,0.526030,0.815543,0.734078,0.715066,0.824318,0.351632,0.932267,0.356276,0.112242
17,0.285220,0.254325,0.906003,0.639921,0.943405,0.743226,0.269261,0.438228,0.263398,0.657945,...,0.202821,0.552089,0.813545,0.757205,0.742489,0.923725,0.120131,0.948478,0.277650,0.126655


In [39]:
# reshape input to be 3D [samples, timesteps, features]
X_train_reshaped = X_train.values.reshape((-1,look_back,n_features))
X_test_reshaped = X_test.values.reshape((-1,look_back,n_features))

y_train_reshaped = y_train.values
y_test_reshaped = y_test.values

In [40]:
print(df.shape[1])

30


In [49]:
X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []

look_back = 8
n_features = 29

for i in bond_list:
    chunk = final_data[i,:,:]
    chunk = chunk.transpose()
    missing_indices = list(set((np.argwhere(np.isnan(chunk))[:,0])))
    all_indices = list(np.arange(0,206))
    all_indices
    non_missing_indices = [x for x in all_indices if x not in missing_indices]
    non_missing_indices
    data = chunk[[non_missing_indices],:]
    data = data.reshape((data.shape[1],data.shape[2]))
    df = pd.DataFrame(data,columns = c)
    # Walk-forward data split to avoid data leakage
    X_train, y_train, X_test, y_test, scale_X = series_to_supervised(df, train_size=0.7, n_in=look_back, n_out=7, target_column='target', dropnan=True, scale_X=True)
    # reshape input to be 3D [samples, timesteps, features]
    X_train_reshaped = X_train.values.reshape((-1,look_back,n_features))
    X_test_reshaped = X_test.values.reshape((-1,look_back,n_features))

    y_train_reshaped = y_train.values
    y_test_reshaped = y_test.values
    
    X_train_list.append(X_train_reshaped)
    X_test_list.append(X_test_reshaped)
    y_train_list.append(y_train_reshaped)
    y_test_list.append(y_test_reshaped)

In [50]:
X_tr = np.vstack(X_train_list)
X_te = np.vstack(X_test_list)
y_tr = np.vstack(y_train_list)
y_te = np.vstack(y_test_list)
X_tr.shape

(4842, 8, 29)

In [51]:
tr = Transformer()
tr.train(X_train_reshaped,y_train_reshaped)
_, rmse_result, mae_result, smape_result, r2_result = tr.evaluate(X_test_reshaped,y_test_reshaped)
print('Result \n RMSE = %.2f  \n MAE = %.2f \n R2 = %.3f' % (rmse_result,mae_result,r2_result))



Epoch 1/200
2/2 [==============================] - ETA: 0s - loss: 0.1768 - rmse: 0.4205 - mae: 0.3669 - smape: 65.0145 - coeff_determination: -111.0068WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,rmse,mae,smape,coeff_determination

Epoch 1: loss improved from inf to 0.17677, saving model to checkpoint/Transformer.best26072023_20:19:13.hdf5
2/2 [==============================] - 7s 256ms/step - loss: 0.1768 - rmse: 0.4205 - mae: 0.3669 - smape: 65.0145 - coeff_determination: -111.0068
Epoch 2/200
2/2 [==============================] - ETA: 0s - loss: 0.1658 - rmse: 0.4072 - mae: 0.3709 - smape: 68.5599 - coeff_determination: -99.8378WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,rmse,mae,smape,coeff_determination

Epoch 2: loss improved from 0.17677 to 0.16581, saving model to checkpoint/Transformer.best26072023_20:19:13.hdf5
2/2 [===========

In [49]:
R2_list = []
look_back = 8
n_features = 29
tr = Transformer()

for i in bond_list:
    chunk = final_data[i,:,:]
    chunk = chunk.transpose()
    missing_indices = list(set((np.argwhere(np.isnan(chunk))[:,0])))
    all_indices = list(np.arange(0,206))
    all_indices
    non_missing_indices = [x for x in all_indices if x not in missing_indices]
    non_missing_indices
    data = chunk[[non_missing_indices],:]
    data = data.reshape((data.shape[1],data.shape[2]))
    df = pd.DataFrame(data,columns = c)
    # Walk-forward data split to avoid data leakage
    X_train, y_train, X_test, y_test, scale_X = series_to_supervised(df, train_size=0.7, n_in=look_back, n_out=7, target_column='target', dropnan=True, scale_X=True)
    # reshape input to be 3D [samples, timesteps, features]
    X_train_reshaped = X_train.values.reshape((-1,look_back,n_features))
    X_test_reshaped = X_test.values.reshape((-1,look_back,n_features))

    y_train_reshaped = y_train.values
    y_test_reshaped = y_test.values

    tr.train(X_train_reshaped,y_train_reshaped)
    _, rmse_result, mae_result, smape_result, r2_result = tr.evaluate(X_test_reshaped,y_test_reshaped)
    print('Result \n RMSE = %.2f  \n MAE = %.2f \n R2 = %.3f' % (rmse_result,mae_result,r2_result))
    R2_list.append(r2_result)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 8, 29)]      0           []                               
                                                                                                  
 layer_normalization_8 (LayerNo  (None, 8, 29)       58          ['input_2[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 multi_head_attention_4 (MultiH  (None, 8, 29)       121885      ['layer_normalization_8[0][0]',  
 eadAttention)                                                    'layer_normalization_8[0][0]']  
                                                                                            